In [24]:
#Libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

#Functions
tfk = tf.keras
tf.keras.backend.set_floatx("float64")
tfd = tfp.distributions

#Helper Functions
scaler = StandardScaler()
detector = IsolationForest(n_estimators=1000, contamination="auto", random_state=0)
neg_log_likelihood = lambda x, rv_x: -rv_x.log_prob(x)

In [17]:
#Load Data
data = pd.read_csv("U235_2.csv")

In [28]:
#Remove row with missing values
columns = ["Mass", "Fission Yield"]
data = data[columns].dropna(axis=0)

In [30]:
X_t = scaler.fit_transform(data)

is_inlier = detector.fit_predict(X_t)
X_t = X_t[(is_inlier > 0),:]
dataset = pd.DataFrame(X_t, columns=columns)

x = data["Mass"]
y = data["Fission Yield"]

In [31]:
n_epochs = 50
n_samples = dataset.shape[0]
n_batches = 10
batch_size = np.floor(n_samples/n_batches)
buffer_size = n_samples

n_train = int(0.7*dataset.shape[0])

data = tf.data.Dataset.from_tensor_slices((dataset[x].values, dataset[y].values))
data = data.shuffle(n_samples, reshuffle_each_iteration=True)

data_train = data.take(n_train).batch(batch_size).repeat(n_epochs)
data_test = data.skip(n_train).batch(1)

KeyError: "None of [Int64Index([ 66,  67,  68,  69,  70,  71,  72,  73,  74,  75,\n            ...\n            163, 164, 165, 166, 167, 168, 169, 170, 171, 172],\n           dtype='int64', length=107)] are in the [columns]"